In [2]:
#Name: CS 458 Person Type Tree visual
#Authors: Liliana Pacheco, Chantelle Suarez, Yan Tarpley
#Date: December 9, 2019
#Description: This code takes in the UsedAutoRELEVATEFfirst10000-noLatLong.csv excel sheet and then processes the data
#by doing the following thins:
#   Removing empty columns and rows
#   converting all non numeric data
#   filling in missing values with the average for the columns
#   removing columns with data that has more than 60% missing values
#This code then goes on to print the Decision Tree for this data set with a target of 
#Person Type. Vehicle Type was analysed and evaluated on a different Code for readibility.

# Load libraries
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

#----------------------------------------------------------------------------------------------#
#                                 May Change File Paths Here                                   #

personDF = pd.read_excel(r"C:\Users\lilia_fdv6j62\Documents\autoData.xlsx", header=0)

#----------------------------------------------------------------------------------------------#


#---------------------------------------------------------------------------------------
#*    This Function was written using the following referrence
#*    Title: Handling Non-Numerical Data for Machine Learning
#*    Author: Harrison@pythonprogramming.net
#*    Date: December 9, 2019
#*    Availability: https://pythonprogramming.net/working-with-non-numerical-data-machine-learning-tutorial/
#*
#---------------------------------------------------------------------------------------
def convertLabels(df):
    columns = df.columns.values
    for column in columns:
        textValues = {}
        def convert(val):
            return textValues[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            contents = df[column].values.tolist()
            uniqueVal = set(contents)
            x = 0
            for unique in uniqueVal:
                if unique not in textValues:
                    textValues[unique] = x
                    x+=1
            df[column] = list(map(convert, df[column])) 
                
    return df
#-----------------------------------------------------------------------------------------

#remove all empty columns in the dataframe
personDF = personDF.dropna(axis = 1, how = 'all')

#Move Target column "Person Type" to the end/right most column
personDF = personDF[personDF['Person Type'].notnull()]
personDFTarget = personDF.pop('Person Type')
personDF['Person Type'] = personDFTarget

#remove columns with more than 60% of data missing
personDF = personDF.loc[:, personDF.isin([' ','NULL',0]).mean() < .4]
#fill in any missing values with the average
personDF.fillna(personDF.mean(), inplace = True)
#Make All Data Numeric
personDf = convertLabels(personDF)

#Split Training Data For Model Analysis
finalData = personDF.values
xVal = finalData[:,0:len(personDF.columns) - 1]
yVal= finalData[:,len(personDF.columns) - 1]
xTrain, xTest, yTrain, yTest = train_test_split(xVal, yVal, test_size=0.20, random_state=1)

    
#Select The Best Model For Testing
print("The best model results in a Decision Tree Model.")
testModel =  DecisionTreeClassifier()
testModel.fit(xTrain, yTrain)

#Print Tree onto PDF
print("Printing Tree...")

from sklearn import tree

tree.export_graphviz(testModel, out_file='tree.dot')

from sklearn.externals.six import StringIO  
import pydot 

dot_data = StringIO()

tree.export_graphviz(testModel, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph[0].write_pdf("Person Type Decision Tree.pdf")
print("Tree Printed.")

The best model results in a Decision Tree Model.
Printing Tree...
Tree Printed.
